In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:
# Creating Spark Session
sp = SparkSession.builder.appName("Customer").getOrCreate()

In [3]:
df_cc = sp.read.format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
    .option("dbtable", "CDW_SAPP_CREDIT_CARD") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

df_cust = sp.read.format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
    .option("dbtable", "CDW_SAPP_CUSTOMER") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

df_br = sp.read.format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
    .option("dbtable", "CDW_SAPP_BRANCH") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

In [4]:
df_cc.show()

+-----------+----------------+---------+--------------+----------------+-----------------+--------+
|BRANCH_CODE|      CUST_CC_NO| CUST_SSN|TRANSACTION_ID|TRANSACTION_TYPE|TRANSACTION_VALUE|  TIMEID|
+-----------+----------------+---------+--------------+----------------+-----------------+--------+
|        156|4210653312478046|123455692|         22562|         Grocery|            91.08|20180813|
|        180|4210653342242023|123451310|         45069|           Bills|            77.79|20180315|
|        114|4210653349028689|123459988|             1|       Education|             78.9|20180214|
|         35|4210653349028689|123459988|             2|   Entertainment|            14.24|20180320|
|        107|4210653342242023|123451310|         45070|      Healthcare|            20.47|20180419|
|        114|4210653312478046|123455692|         22563|           Bills|             22.2|20180626|
|        160|4210653349028689|123459988|             3|         Grocery|             56.7|20180708|


In [8]:
df_join = df_cc.join(df_cust, df_cc.CUST_SSN == df_cust.SSN, 'inner').select(col('SSN'), col('CUST_ZIP'), col('TIMEID'), \
      col('TRANSACTION_ID'), col('TRANSACTION_VALUE'))

In [9]:
df_join.show()

+---------+--------+--------+--------------+-----------------+
|      SSN|CUST_ZIP|  TIMEID|TRANSACTION_ID|TRANSACTION_VALUE|
+---------+--------+--------+--------------+-----------------+
|123455659|   10954|20180723|         22772|            97.57|
|123455659|   10954|20180625|         22773|            65.32|
|123455659|   10954|20181022|         22774|             5.76|
|123455659|   10954|20180521|         22775|           100.81|
|123455659|   10954|20180726|         22776|            57.93|
|123455659|   10954|20180513|         22777|            76.18|
|123455659|   10954|20181119|         22778|            87.72|
|123455659|   10954|20180228|         22779|            25.73|
|123455659|   10954|20181002|         22780|            58.56|
|123455659|   10954|20180105|         22781|             75.2|
|123455659|   10954|20181207|         22782|            70.19|
|123455659|   10954|20180827|         22783|            85.66|
|123455659|   10954|20180908|         22784|           

In [10]:
df_join.createOrReplaceTempView('CC_CUST')

In [ ]:
# data = sp.sql('select * from CC_CUST where CUST_ZIP == "53066" and substr(TIMEID,1,4) == "2018" and substr(TIMEID,5,2)=="06"')
# data.show(100)

In [11]:
def trans_value(zip,yr,mm):
    print("inside the function", zip, yr, mm)
    data = sp.sql('select distinct * from CC_CUST where CUST_ZIP == "{}" and substr(TIMEID,1,4) == "{}" \
        and substr(TIMEID,5,2) == "{}" ORDER BY substr(TIMEID,7,2) DESC'.format(zip,yr,mm))
    data.show()


zip = str(input("Enter 5-digit zipcode: "))
yr = str(input("Enter year: "))
mm = str(input("Enter month: "))
trans_value(zip,yr,mm)

inside the function 53066 2018 06
+---------+--------+--------+--------------+-----------------+
|      SSN|CUST_ZIP|  TIMEID|TRANSACTION_ID|TRANSACTION_VALUE|
+---------+--------+--------+--------------+-----------------+
|123458614|   53066|20180628|          6691|             19.2|
|123457562|   53066|20180628|         12436|            65.84|
|123457286|   53066|20180627|         13783|            42.01|
|123451357|   53066|20180626|         44896|            13.47|
|123458614|   53066|20180625|          6697|             6.66|
|123457286|   53066|20180623|         13824|            52.27|
|123454357|   53066|20180622|         29654|            82.03|
|123457286|   53066|20180622|         13799|            41.65|
|123457562|   53066|20180621|         12400|            17.85|
|123457562|   53066|20180621|         12387|            21.05|
|123454391|   53066|20180620|         29420|             82.4|
|123452342|   53066|20180618|         40354|            38.26|
|123451357|   53066|2

Functional requirements 2.1 (2)

In [ ]:
# df_cc.select('transaction_type','transaction_value').groupby('transaction_type').agg(count('transaction_type'), \
    # sum('transaction_value')).show()

In [5]:
df_cc.createOrReplaceTempView('cdw_cc')

In [6]:
data1 = sp.sql('SELECT transaction_type, COUNT(*) as trans_count, \
                round(SUM(transaction_value),2) as trans_val \
                FROM cdw_cc \
                GROUP BY transaction_type')

In [7]:
data1.show()

+----------------+-----------+---------+
|transaction_type|trans_count|trans_val|
+----------------+-----------+---------+
|       Education|       6638|337980.07|
|   Entertainment|       6635| 338950.1|
|      Healthcare|       6723| 340476.2|
|         Grocery|       6549|337051.63|
|            Test|       6683|341310.37|
|             Gas|       6605|336059.26|
|           Bills|       6861|351405.28|
+----------------+-----------+---------+



Functional requirements 2.1 (3)

In [ ]:
data_br = df_cc.join(df_br, df_br.BRANCH_CODE == df_cc.BRANCH_CODE, 'inner').select(df_br['branch_code'].alias('BC1'), \
            col('branch_name'), col('branch_state'), col('transaction_value'))
data_br.show()

In [ ]:
data_br.createOrReplaceTempView('data_br_view')

In [12]:
def trans_by_branch(state):
    data1 = sp.sql("SELECT bc1, COUNT(bc1), \
                round(SUM(transaction_value),2) \
                FROM data_br_view \
                WHERE BRANCH_STATE == '{}' \
                GROUP BY bc1 \
                ORDER BY bc1".format(state))
    data1.show()

state = str(input("Enter State : "))
trans_by_branch(state)

AnalysisException: Table or view not found: data_br_view; line 1 pos 93;
'Sort ['bc1 ASC NULLS FIRST], true
+- 'Aggregate ['bc1], ['bc1, unresolvedalias('COUNT('bc1), None), unresolvedalias('round('SUM('transaction_value), 2), None)]
   +- 'Filter ('BRANCH_STATE = tx)
      +- 'UnresolvedRelation [data_br_view], [], false
